## Here, we are training a StyleGAN2 model from scratch to compare to StyleGAN2-ADA on the same dataset

In [2]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile
import zipfile
import subprocess
import sys
import dotenv
import os

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

dotenv.load_dotenv(override=True)
WORK = os.getenv('WORK')
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/stylegan2')
import DeepFAMS

WORK, PROJ_DIR = DeepFAMS.utils.set_env()

Tensorflow version: 1.14.0


In [3]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(
    PROJ_DIR, 'AFHQ-WILD')

In [4]:
RAW_IMGS_DIR_ = f'{PROJ_DIR}/datasets/AFHQ_images_raw/afhq/train/wild'
TRAIN_RUNS_DIR = f'{PROJ_DIR}/training_runs/StyleGAN2_WILD-AFHQ_training-runs'

In [5]:
raw_imgs = glob(f'{RAW_IMGS_DIR_}/*')
print(len(raw_imgs))

4738


In [ ]:
# for x in tqdm(raw_imgs):
#     DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

In [6]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 4738, Resized: 4738


In [ ]:
# DeepFAMS.preprocessing.tf_record_exporter(
#     tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

In [8]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/6.1
module load cuda/10.0
$WORK/.conda/envs/stylegan2/bin/python3 $WORK/ADA_Project/stylegan2/run_training.py \
    --num-gpus=2 \
    --data-dir=$WORK/ADA_Project/datasets \
    --result-dir={TRAIN_RUNS_DIR} \
    --config=config-f \
    --mirror-augment=true \
    --total-kimg=1 \
    --dataset=AFHQ-WILD_custom''', PROJ_DIR)

Local submit - run_dir: /work/chaselab/malyetama/ADA_Project/training_runs/StyleGAN2_WILD-AFHQ_training-runs/00003-stylegan2-AFHQ-WILD_custom-2gpu-config-f

dnnlib: Running training.training_loop.training_loop() on localhost...

/work/chaselab/malyetama/.conda/envs/stylegan2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

  _np_qint8 = np.dtype([("qint8", np.int8, 1)])

/work/chaselab/malyetama/.conda/envs/stylegan2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

/work/chaselab/malyetama/.conda/envs/stylegan2/lib/python3.6/site-packages/tensorflow/python/framework/dtype

Building TensorFlow graph...

Initializing logs...

Training for 1 kimg...



tick 0     kimg 0.1      lod 0.00  minibatch 32   time 34s          sec/tick 34.1    sec/kimg 266.65  maintenance 0.0    gpumem 6.1

Cache dir: .stylegan2-cache


Cache dir path: .stylegan2-cache

network-snapshot-000000        time 7m 40s       fid50k 325.5382

tick 1     kimg 1.0      lod 0.00  minibatch 32   time 9m 46s       sec/tick 55.1    sec/kimg 61.47   maintenance 496.7  gpumem 6.1

network-snapshot-000001        time 6m 07s       fid50k 325.8816

dnnlib: Finished training.training_loop.training_loop() in 18m 25s.




In [5]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ADA_Project/training_runs/StyleGAN2_WILD-AFHQ_training-runs/00006-stylegan2-AFHQ-WILD_custom-2gpu-config-f/network-snapshot-003064.pkl


In [16]:
DeepFAMS.utils.executePopen(f'''#!/bin/bash
module load anaconda
module load compiler/gcc/6.1
module load cuda/10.0
$WORK/.conda/envs/stylegan2/bin/python3 $WORK/ADA_Project/stylegan2/run_training.py \
    --num-gpus=2 \
    --data-dir=$WORK/ADA_Project/datasets \
    --result-dir={TRAIN_RUNS_DIR} \
    --config=config-f \
    --mirror-augment=true \
    --dataset=AFHQ-WILD_custom
    --resume-pkl={latest_snap}''', PROJ_DIR)

Local submit - run_dir: /work/chaselab/malyetama/ADA_Project/training_runs/StyleGAN2_WILD-AFHQ_training-runs/00004-stylegan2-AFHQ-WILD_custom-2gpu-config-f

dnnlib: Running training.training_loop.training_loop() on localhost...

/work/chaselab/malyetama/.conda/envs/stylegan2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

  _np_qint8 = np.dtype([("qint8", np.int8, 1)])

/work/chaselab/malyetama/.conda/envs/stylegan2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])

/work/chaselab/malyetama/.conda/envs/stylegan2/lib/python3.6/site-packages/tensorflow/python/framework/dtype

KeyboardInterrupt: 